In [2]:
!pip install lightgbm

  Using cached lightgbm-4.6.0-py3-none-win_amd64.whl.metadata (17 kB)
Using cached lightgbm-4.6.0-py3-none-win_amd64.whl (1.5 MB)


In [1]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from datetime import datetime

# === 1. Cargar el modelo entrenado ===
with open("modelo_lightgbm_final.pkl", "rb") as f:
    model = pickle.load(f)

# === 2. Cargar la nueva data de predicción ===
df_pred = pd.read_parquet("df_pt_origin.parquet")
df_pred.columns = df_pred.columns.str.lower()

# Procesamiento mínimo como antes
df_pred.fillna(0, inplace=True)
df_pred["codzona_x"] = df_pred["codzona_x"].replace("", -1).astype(int)
df_pred["codregion"] = df_pred["codregion"].replace("", -1).astype(int)
df_pred = df_pred.astype({
    "codterritorio": int,
    "codzona_x": int,
    "codregion": int,
    "aniocampanaingreso": int,
    "campanaingreso": int,
    "aniocampana": int,
})

# === 3. Preparar X_test ===
X_test = df_pred.drop(["codvendedora", "clase", "aniocampana"], axis=1)
X_test = X_test.apply(pd.to_numeric, errors='coerce').fillna(0)

# === 4. Predecir probabilidades ===
probs = model.predict(X_test)
df_pred["probs"] = probs

# === 5. Clasificar en deciles y riesgo ===
df_pred["proba_decil"] = pd.qcut(
    df_pred["probs"].rank(method="first"),
    10,
    labels=False
)

df_pred["Perfil_Riesgo"] = np.where(
    df_pred.proba_decil < 5, "1 - Riesgo Bajo",
    np.where(df_pred.proba_decil < 8, "2 - Riesgo Medio", "3 - Riesgo Alto")
)

# === 6. Guardar resultados ===
fecha_str = datetime.now().strftime("%Y%m%d_%H%M%S")
result_cols = ["aniocampana", "codvendedora", "codregion", "probs", "Perfil_Riesgo"]
output_path = f"predicciones_resultado_{fecha_str}.csv"

df_pred[result_cols].to_csv(output_path, index=False)
print(f"✅ Resultados guardados en {output_path}")


[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
✅ Resultados guardados en predicciones_resultado_20250715_210037.csv


In [2]:
df_pred.head()

,aniocampanaproceso,aniocampana,codvendedora,codterritorio,codcomportamientorolling,edad,flagcorreovalidado,flagcelular,flagdigital,desestadocivil_cat,...,flag19,flag20,desv_pedmin,desv_pedmin_adj,clase,target_cobranza31_ca,prom_dias_pago,probs,proba_decil,Perfil_Riesgo
0,202507,202507,053454100,2428,1,33,0,1,0,5,...,0,0,195.14000,181.02040,0,0,21.00000,1,7,2 - Riesgo Medio
1,202507,202507,053438091,10285,1,23,0,1,0,5,...,0,0,-7.99000,-10.16540,1,1,21.00000,0,0,1 - Riesgo Bajo
5799,202507,202507,050258629,2428,1,32,0,1,0,5,...,0,0,-7.11000,-7.11000,0,0,189.00000,1,8,3 - Riesgo Alto
8246,202507,202507,053524966,2428,1,21,0,1,0,5,...,0,0,134.18000,134.18000,0,0,189.00000,1,8,3 - Riesgo Alto
49929,202507,202507,052367042,2428,6,20,0,1,0,5,...,0,1,199.51000,171.99140,0,0,23.50000,0,0,1 - Riesgo Bajo
